In [10]:
import numpy as np
import time

In [14]:
def computeInterCluster(R):
    iloczynySkalarne = -2*np.dot(R.T, R)
    kwadratyDlugosci = np.sum(R**2, axis=0, keepdims=True)
    iloczynySkalarne += kwadratyDlugosci
    iloczynySkalarne += kwadratyDlugosci.T

    return iloczynySkalarne

def computeIntraClusterDistances(data, C, R, I, K, N):
    swojeCentra = np.take(R, C, axis=1)
    swojeCentra -= data
    swojeCentra **= 2

    intraClusterDistances = np.sqrt(np.sum(swojeCentra, axis=0, keepdims=True))

    liczebnosc = np.zeros(K)
    intraAvg = np.zeros(K)
    intraMax = np.zeros(K)
    
    for i in range(N):
        intraAvg[C[i]] += intraClusterDistances[0][i]
        intraMax[C[i]] = max(intraClusterDistances[0][i], intraMax[C[i]])
        liczebnosc[C[i]] += 1
    
    intraAvg /= np.maximum(liczebnosc, 1)
    
    return (intraAvg, intraMax)
        
    

def KMeans(K, data, printInfo=False):
    d = np.size(data, axis=0)
    N = np.size(data, axis=1)
    I = np.eye(K)
    
    #Centra grup (losowe wektory z danych)
    #R = np.take(data, np.random.choice(np.arange(0,N), K, replace=False), axis=1)
    R = np.empty([d,K])
    choices = np.random.choice(np.arange(0,N), K, replace=False)
    for i in range(K):
        R[:,i] = data[:, choices[i]]
    #Przynależnosc do grupy (na poczatku wszyscy do zero)
    C = np.zeros((1,N), dtype=np.int64)
    macierPrzynaleznosci = None

    iteration = 0
    groupsChanged = True
    while groupsChanged:
        iteration += 1
        #Interesuje nas minimalna wartość <r,r> -2<u,r>, gdzie u to wektor z danych a r to jakies centrum
        iloczynySkalarne = -2. * np.dot(data.T, R)
        R**=2
        kwadratyDlugosciR = np.sum(R, axis=0, keepdims=True)
        iloczynySkalarne += kwadratyDlugosciR

        #Dla kazdego wektora z danych wybieramy najblizszy wektor z R i aktualizujemy grupy
        newC = np.argmin(iloczynySkalarne, axis=1)
        groupsChanged = not np.array_equal(C, newC)
        C = newC

        #Obliczamy srodki ciezkosci dla kazdej grupy
        macierzPrzynaleznosci = np.take(I, C, axis=0)
        #Sumy danych w każdej z grup
        R = np.dot(data, macierzPrzynaleznosci)
        liczebnosciGrup = np.maximum(np.sum(macierzPrzynaleznosci, axis=0, keepdims=True), 1)
        R /= liczebnosciGrup

    #Liczymy wielkosci grup, odleglosci miedzy grupami, srednie i najwieksze odleglosci w grupach
    groupSizes = np.sum(macierzPrzynaleznosci, axis=0).astype(np.int64)
    interCluster = computeInterCluster(R)
    intraClusterAvg, intraClusterMax = computeIntraClusterDistances(data, C, R, I, K, N)
    
    if printInfo:
        print("Iterations:", iteration, "\n")
        print("Group sizes:\n", groupSizes, "\n")
        print("Inter cluster distances:")
        print(interCluster, "\n")
        print("Intra cluster average distance to center\n", intraClusterAvg, "\n")
        print("Intra cluster maximum distance to center\n", intraClusterMax, "\n")
    
    return (C, R, interCluster, intraClusterAvg, intraClusterMax)

In [15]:
#Wczytujemy wszystkie transakcje do listy
transactions = [[int(id) for id in line.split()] for line in open("kosarak.dat")]

In [16]:
occurences = {}

for i in range(len(transactions)):
    for j in range(len(transactions[i])):
        if not transactions[i][j] in occurences:
            occurences[transactions[i][j]] = 0
        occurences[transactions[i][j]] += 1
        
mostPopular = sorted(occurences, key=occurences.__getitem__, reverse=True)

In [32]:
def getDataForTMostPopular(transactions, mostPopular, occurences, T):
    order = {mostPopular[i] : i for i in range(T)}
    
    p = np.zeros((T,T))
    
    for i in range(len(transactions)):
        fromTMostPop = [order[id] for id in transactions[i] if id in order]
        for j in fromTMostPop:
            for k in fromTMostPop:
                p[j][k] += 1 if j != k else 0
    
    for id, orderPos in order.items():
        p[orderPos][orderPos] = occurences[id]
    
    return p, order

In [34]:
T = 1000
data, order = getDataForTMostPopular(transactions, mostPopular, occurences, T)
data = data.T
print("Data generated")

Data generated


# T = 1000

## Mała liczba grup - 4
Wyodrębniają się 2 grupy: 3 i 5

In [35]:
groups, centers, interCluster, intraClusterAvg, intraClusterMax = KMeans(4, data, printInfo=True)

Iterations: 24 

Group sizes:
 [953  39   3   5] 

Inter cluster distances:
[[  1.63912773e-07   9.24731657e+08   3.55341197e+11   2.32014400e+10]
 [  9.24731657e+08   3.33786011e-06   3.23344629e+11   1.52949539e+10]
 [  3.55341197e+11   3.23344629e+11  -7.44628906e-03   2.17669041e+11]
 [  2.32014400e+10   1.52949539e+10   2.17669041e+11   6.86645508e-05]] 

Intra cluster average distance to center
 [   5291.21044941   26925.35782938  211834.28539285   89036.9394518 ] 

Intra cluster maximum distance to center
 [  28290.48440561   84505.96533093  246199.8386079   157325.97995436] 



## Mała liczba grup - 5
Dalej mamy 3 i 5

In [36]:
groups, centers, interCluster, intraClusterAvg, intraClusterMax = KMeans(5, data, printInfo=True)

Iterations: 24 

Group sizes:
 [ 39   3 952   5   1] 

Inter cluster distances:
[[  3.33786011e-06   3.23344629e+11   9.24654844e+08   1.52949539e+10
    1.05002727e+09]
 [  3.23344629e+11  -7.44628906e-03   3.55341473e+11   2.17669041e+11
    3.55130535e+11]
 [  9.24654844e+08   3.55341473e+11  -1.49011612e-08   2.32011234e+10
    5.22245318e+07]
 [  1.52949539e+10   2.17669041e+11   2.32011234e+10   6.86645508e-05
    2.35549809e+10]
 [  1.05002727e+09   3.55130535e+11   5.22245318e+07   2.35549809e+10
    0.00000000e+00]] 

Intra cluster average distance to center
 [  26925.35782938  211834.28539285    5289.55686427   89036.9394518
       0.        ] 

Intra cluster maximum distance to center
 [  84505.96533093  246199.8386079    28291.36344979  157325.97995436
       0.        ] 



In [39]:
groups, centers, interCluster, intraClusterAvg, intraClusterMax = KMeans(10, data, printInfo=True)

Iterations: 22 

Group sizes:
 [ 39  16   5  34  22   3 342 101  24 414] 

Inter cluster distances:
[[  3.57627869e-07   3.05856190e+08   1.91317277e+10   1.67746229e+08
    6.99857396e+08   3.41022284e+11   2.79059644e+08   1.16762166e+08
    2.90379139e+08   3.99145802e+08]
 [  3.05856190e+08   0.00000000e+00   2.27325920e+10   1.33095270e+08
    1.45383529e+09   3.53528291e+11   1.02155596e+08   1.13625974e+08
    1.90918071e+08   1.26563219e+08]
 [  1.91317277e+10   2.27325920e+10   6.86645508e-05   2.12954295e+10
    1.35708075e+10   2.17669041e+11   2.32791144e+10   2.18298260e+10
    2.15376247e+10   2.39475515e+10]
 [  1.67746229e+08   1.33095270e+08   2.12954295e+10  -1.19209290e-07
    1.01954073e+09   3.47474224e+11   6.21326635e+07   4.69017764e+07
    7.39676185e+07   9.18923988e+07]
 [  6.99857396e+08   1.45383529e+09   1.35708075e+10   1.01954073e+09
   -1.28746033e-05   3.14110311e+11   1.52223199e+09   1.19436027e+09
    1.15610301e+09   1.68961604e+09]
 [  3.41022284e

In [40]:
groups, centers, interCluster, intraClusterAvg, intraClusterMax = KMeans(15, data, printInfo=True)

Iterations: 58 

Group sizes:
 [  7 104  11 357  81  17 232  26   5  16  70  24  12   3  35] 

Inter cluster distances:
[[ -3.81469727e-06   3.73395062e+09   3.55363216e+09   3.52583160e+09
    2.90396201e+09   3.57534638e+09   3.29464490e+09   2.51753214e+09
    1.08057253e+10   1.18592501e+09   3.53344157e+09   3.05924854e+09
    1.75780422e+09   2.95224032e+11   2.36106351e+09]
 [  3.73395062e+09  -6.51925802e-09   3.37771285e+07   1.08987377e+07
    1.38981902e+08   3.87702466e+07   4.14272051e+07   1.47421790e+08
    2.43689381e+10   1.14965152e+09   7.63851362e+06   6.98549850e+07
    6.97337072e+08   3.58791977e+11   4.22592230e+08]
 [  3.55363216e+09   3.37771285e+07  -1.49011612e-08   2.51232824e+07
    1.03219015e+08   3.59237374e+07   3.67482826e+07   1.25031440e+08
    2.36911100e+10   1.01134273e+09   3.25175715e+07   7.52369766e+07
    6.39280290e+08   3.57199443e+11   3.34978408e+08]
 [  3.52583160e+09   1.08987377e+07   2.51232824e+07  -1.49011612e-08
    7.48222137e+07

# T = 500
Nic mi się tutaj nie udało wyłowić

In [41]:
groups, centers, interCluster, intraClusterAvg, intraClusterMax = KMeans(5, data[:500,:500], printInfo=True)

Iterations: 17 

Group sizes:
 [  3  15   5 443  34] 

Inter cluster distances:
[[ -2.80761719e-03   3.52719522e+11   2.17513778e+11   3.52049900e+11
    3.20468058e+11]
 [  3.52719522e+11   5.96046448e-07   2.25038017e+10   9.81723205e+07
    1.00861176e+09]
 [  2.17513778e+11   2.25038017e+10   9.15527344e-05   2.22460291e+10
    1.47231632e+10]
 [  3.52049900e+11   9.81723205e+07   2.22460291e+10  -1.49011612e-07
    8.82292734e+08]
 [  3.20468058e+11   1.00861176e+09   1.47231632e+10   8.82292734e+08
   -7.15255737e-06]] 

Intra cluster average distance to center
 [ 211705.11722792    2474.32880326   88787.36930363    6543.20831978
   28148.26600687] 

Intra cluster maximum distance to center
 [ 245981.60172889    4908.10368721  157254.07679319   27394.56553453
   83246.78336533] 



In [42]:
groups, centers, interCluster, intraClusterAvg, intraClusterMax = KMeans(10, data[:500,:500], printInfo=True)

Iterations: 36 

Group sizes:
 [  7   5 111  14 325  13   3  20   1   1] 

Inter cluster distances:
[[ -2.86102295e-06   1.07024319e+10   2.65919192e+09   1.79810560e+09
    3.25094551e+09   3.23772500e+09   2.94761798e+11   1.29958553e+09
    3.35563615e+09   3.40743364e+09]
 [  1.07024319e+10   9.15527344e-05   2.08526819e+10   1.79632257e+10
    2.27921821e+10   2.25390167e+10   2.17513778e+11   1.59934738e+10
    2.31910242e+10   2.33408982e+10]
 [  2.65919192e+09   2.08526819e+10  -2.98023224e-07   3.27384813e+08
    7.00078013e+07   1.00723451e+08   3.47220213e+11   3.93649683e+08
    1.68267977e+08   1.77041818e+08]
 [  1.79810560e+09   1.79632257e+10   3.27384813e+08  -2.38418579e-07
    4.38831334e+08   4.79211782e+08   3.34948849e+11   3.68776910e+08
    5.12147100e+08   5.29479869e+08]
 [  3.25094551e+09   2.27921821e+10   7.00078013e+07   4.38831334e+08
    2.98023224e-08   5.88235783e+07   3.53909411e+11   7.32442211e+08
    5.28447302e+07   5.34262055e+07]
 [  3.23772500e

In [44]:
groups, centers, interCluster, intraClusterAvg, intraClusterMax = KMeans(15, data[:500,:500], printInfo=True)

Iterations: 29 

Group sizes:
 [ 14   5   5  15  16  10  77  91  34  34   3   1 179   1  15] 

Inter cluster distances:
[[ -1.43051147e-06   1.53784147e+10   1.12586937e+09   3.99995172e+08
    8.55539505e+08   3.91498707e+08   6.43217537e+08   9.86884312e+08
    3.57627728e+08   7.08713233e+08   3.24238955e+11   7.64680252e+09
    8.90347365e+08   1.07592679e+09   8.97450226e+08]
 [  1.53784147e+10   9.15527344e-05   1.09160361e+10   1.95545093e+10
    2.26419818e+10   1.63194219e+10   2.15527129e+10   2.31049927e+10
    1.95228596e+10   2.17855016e+10   2.17513778e+11   1.52751444e+10
    2.28315115e+10   2.33408982e+10   2.25038017e+10]
 [  1.12586937e+09   1.09160361e+10  -1.90734863e-06   2.13288848e+09
    3.17620188e+09   1.52424388e+09   2.87505065e+09   3.34241940e+09
    2.32140356e+09   2.84667486e+09   2.96148038e+11   6.40525171e+09
    3.27539395e+09   3.41727054e+09   3.21747675e+09]
 [  3.99995172e+08   1.95545093e+10   2.13288848e+09  -1.66893005e-06
    1.50486394e+08

# T = 200
Wyłania się kolejna grupa - 7

In [45]:
groups, centers, interCluster, intraClusterAvg, intraClusterMax = KMeans(5, data[:200,:200], printInfo=True)

Iterations: 12 

Group sizes:
 [158   5  27   3   7] 

Inter cluster distances:
[[  0.00000000e+00   2.08129417e+10   3.89027420e+08   3.46749969e+11
    2.68538555e+09]
 [  2.08129417e+10   0.00000000e+00   1.58323774e+10   2.17229513e+11
    1.05005274e+10]
 [  3.89027420e+08   1.58323774e+10   2.38418579e-07   3.26476844e+11
    1.26304739e+09]
 [  3.46749969e+11   2.17229513e+11   3.26476844e+11  -1.22070312e-04
    2.93836590e+11]
 [  2.68538555e+09   1.05005274e+10   1.26304739e+09   2.93836590e+11
   -4.76837158e-06]] 

Intra cluster average distance to center
 [   8487.12092134   88363.60487011   20035.68864784  211433.44144097
   45083.13950544] 

Intra cluster maximum distance to center
 [  26149.92830192  157112.22917914   34386.98405263  245495.63034672
   68486.976009  ] 



In [48]:
groups, centers, interCluster, intraClusterAvg, intraClusterMax = KMeans(10, data[:200,:200], printInfo=True)

Iterations: 19 

Group sizes:
 [46 68  5 33 10  7  3  8 17  3] 

Inter cluster distances:
[[  2.98023224e-08   3.32139945e+07   2.13111961e+10   1.42855793e+08
    3.50779067e+08   2.79789089e+09   3.48164597e+11   3.76745772e+07
    5.91743232e+08   4.23817811e+08]
 [  3.32139945e+07   5.96046448e-08   2.10575345e+10   6.16270819e+07
    4.11743738e+08   2.79986561e+09   3.47909624e+11   8.22649942e+07
    5.34343773e+08   4.62953995e+08]
 [  2.13111961e+10   2.10575345e+10   0.00000000e+00   1.92316593e+10
    1.76363406e+10   1.05005274e+10   2.17229513e+11   2.25919955e+10
    1.54448170e+10   1.86608738e+10]
 [  1.42855793e+08   6.16270819e+07   1.92316593e+10   0.00000000e+00
    3.65232897e+08   2.28993945e+09   3.41286895e+11   2.59326385e+08
    3.01208583e+08   4.52710538e+08]
 [  3.50779067e+08   4.11743738e+08   1.76363406e+10   3.65232897e+08
    1.43051147e-06   1.88461320e+09   3.34665123e+11   5.33359879e+08
    4.19627431e+08   5.34150677e+08]
 [  2.79789089e+09   2.79

In [47]:
groups, centers, interCluster, intraClusterAvg, intraClusterMax = KMeans(15, data[:200,:200], printInfo=True)

Iterations: 15 

Group sizes:
 [14  5  3 53 19  6  5  4 10 29  7  8 29  2  6] 

Inter cluster distances:
[[  5.96046448e-08   1.11906703e+08   3.47827458e+11   9.45048502e+07
    7.77818842e+07   3.39221276e+08   2.10713809e+10   1.34735920e+08
    6.57380221e+08   1.87683983e+08   2.79793842e+09   7.11820161e+08
    6.13071899e+07   3.88861839e+08   3.93687985e+08]
 [  1.11906703e+08   0.00000000e+00   3.47470990e+11   1.84927469e+07
    3.98173515e+07   2.60417284e+08   2.09702358e+10   1.04122309e+08
    4.65068743e+08   5.59474955e+07   2.77914268e+09   6.54321362e+08
    6.15060049e+07   3.92269450e+08   6.32525007e+08]
 [  3.47827458e+11   3.47470990e+11  -1.22070312e-04   3.48482229e+11
    3.44925325e+11   3.32840754e+11   2.17229513e+11   3.42726074e+11
    3.27737788e+11   3.42970971e+11   2.93836590e+11   3.21614074e+11
    3.50198961e+11   3.36455941e+11   3.31105664e+11]
 [  9.45048502e+07   1.84927469e+07   3.48482229e+11  -5.96046448e-08
    2.66906547e+07   2.69692368e+